In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [2]:
fake = pd.read_csv('fake.csv')
true = pd.read_csv('true.csv')


In [3]:
fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class']=0
true['class']=1


In [6]:
data = pd.concat([fake,true],axis=0)


In [8]:
data.sample(10)



,title,text,subject,date,class
16301,Zimbabwe's opposition leader Tsvangirai back i...,HARARE (Reuters) - Zimbabwe opposition leader ...,worldnews,"October 27, 2017",1
12207,KID ROCK HAS A NEW Line Of Trump Apparel…And Y...,I grew up in the same hometown in rural Michig...,politics,"Dec 6, 2016",0
13394,HUMILIATING: Democrats Use RUSSIAN WARSHIPS As...,Maybe Hillary s Russian uranium deal included ...,politics,"Jul 29, 2016",0
13671,WHAT IF The Presidential Race Was A NASCAR Event?,If booster and sponsors were shown in presiden...,politics,"Jun 19, 2016",0
20599,BREAKING: FLINT’S DEMOCRAT MAYOR SUED After Wh...,Democrats looking for attention and along with...,left-news,"May 10, 2016",0
19027,TUCKER CARLSON Defends Trump’s Wiretapping All...,FOX and Friends spoke with Tucker Carlson this...,left-news,"Mar 6, 2017",0
6823,Buffalo Bills Coach Gives Donald Trump The We...,"Buffalo Bills coach Rex Ryan has one job, to i...",News,"April 19, 2016",0
5749,U.S. Senate confirms Tillerson as secretary of...,WASHINGTON (Reuters) - The U.S. Senate confirm...,politicsNews,"February 1, 2017",1
8970,This Anti-Government Oregon Terrorist Took Th...,One of the ringleaders of the terror-minded mi...,News,"January 4, 2016",0
2300,Stephen King Is BRUTALLY Making Fun Of Trump ...,With Donald Trump s intensely bizarre and craz...,News,"March 4, 2017",0


In [10]:
data = data.drop(["title","subject","date"],axis=1)


In [14]:
data.reset_index(inplace=True)

In [15]:
data.drop(['index'],axis = 1,inplace=True)

In [16]:
data.sample(7)

,text,class
10755,AUSTIN A shoving match and war of words near...,0
19656,,0
21904,The gun grabbers are at it again pushing for m...,0
6121,It appears that Republicans aren t quite sure ...,0
38901,SEOUL (Reuters) - U.S. President Donald Trump ...,1
263,A text message which was sent in January to U....,0
10075,This is mob rule AGAIN! The best part of this ...,0


In [19]:
import re
import string

def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [20]:
data["text"] = data["text"].apply(clean_text)


In [24]:
x=data["text"]
y=data["class"]

xtrain,xtest,ytrain,ytest =train_test_split(x,y,test_size=0.25,random_state=42)



In [25]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)


In [26]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [27]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9856570155902005

In [28]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5895
           1       0.98      0.99      0.98      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [30]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']